# EDA de energyco2

In [1]:
import numpy as np
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import Levenshtein as lev
from google.cloud import bigquery
from google.oauth2 import service_account

In [30]:
credentials = service_account.Credentials.from_service_account_file('tidy-hold-359719-f6808bf80ee8.json') #credenciales
project_id = 'tidy-hold-359719' #nombre del proyecto
client = bigquery.Client(credentials= credentials,project=project_id) #creo cliente
dataset_ref = client.dataset("proyecto", project=project_id) #referencia al dataset
dataset = client.get_dataset(dataset_ref) #dataset
#QUERYS
q_NDC="""SELECT * FROM `tidy-hold-359719.proyecto.NDC`"""   #query para tabla NDC
q_energyco2 = """
SELECT Country AS Pais, Year AS Anio, CO2_emission AS Emisiones_de_CO2
FROM `tidy-hold-359719.proyecto.energyco2`
WHERE Energy_type = 'all_energy_types'
"""
NDC = client.query(q_NDC).to_dataframe() #dataframe de NDC
tabla = client.query(q_energyco2).to_dataframe() 

In [15]:
def traductor(df):
  for i in range(len(df)):
    for j in range(len(NDC)):
        if lev.ratio(df['Pais'][i], NDC['Pais'][j]) > 0.6:
            df.Pais[i]= NDC['Pais'][j]
  df.loc[df['Pais']=='World', 'Pais']='Mundo'
  df.loc[df['Pais']=='United States', 'Pais']='Estados Unidos'
  df.loc[df['Pais']=='Dominican Republic', 'Pais']='República Dominicana'
  df.loc[df['Pais']=='Saint Kitts and Nevis', 'Pais']='San Cristóbal y Nieves'
  return df
  
def filtro_latam(df):
  df = df[df.Pais.isin(NDC.Pais.unique().tolist())]
  df.reset_index(drop=True, inplace=True)
  return df

In [6]:
# Importamos y mostramos
tabla = pd.read_csv('energyco2.csv', index_col=0)
tabla.head(12)

,Country,Energy_type,Year,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission
0,World,all_energy_types,1980,292.899790,296.337228,27770.910281,4.298127e+06,68.145921,10.547,4946.627130
1,World,coal,1980,78.656134,80.114194,27770.910281,4.298127e+06,68.145921,10.547,1409.790188
2,World,natural_gas,1980,53.865223,54.761046,27770.910281,4.298127e+06,68.145921,10.547,1081.593377
3,World,petroleum_n_other_liquids,1980,132.064019,133.111109,27770.910281,4.298127e+06,68.145921,10.547,2455.243565
4,World,nuclear,1980,7.575700,7.575700,27770.910281,4.298127e+06,68.145921,10.547,0.000000
5,World,renewables_n_other,1980,20.702344,20.775178,27770.910281,4.298127e+06,68.145921,10.547,0.000000
6,Afghanistan,all_energy_types,1980,0.026583,0.072561,NaN,1.335650e+04,1.990283,0.000,NaN
7,Afghanistan,coal,1980,0.002479,0.002355,NaN,1.335650e+04,1.990283,0.000,NaN
8,Afghanistan,natural_gas,1980,0.002094,0.062820,NaN,1.335650e+04,1.990283,0.000,NaN
9,Afghanistan,petroleum_n_other_liquids,1980,0.014624,0.000000,NaN,1.335650e+04,1.990283,0.000,NaN


In [11]:
# Describimos la tabla
tabla.describe()

,Anio,Emisiones_de_CO2
count,9240.000000,8371.000000
mean,1999.500000,242.933187
std,11.544021,1895.936127
min,1980.000000,0.000000
25%,1989.750000,0.771434
50%,1999.500000,5.384853
75%,2009.250000,48.436207
max,2019.000000,35584.933498


In [12]:
# Vemos información
tabla.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Pais              9240 non-null   object 
 1   Anio              9240 non-null   Int64  
 2   Emisiones_de_CO2  8371 non-null   float64
dtypes: Int64(1), float64(1), object(1)
memory usage: 225.7+ KB


In [9]:
tabla.head()

,Pais,Anio,Emisiones_de_CO2
0,Chad,1988,0.2
1,Chad,1989,0.2
2,Chad,1990,0.2
3,Chad,1991,0.3
4,Chad,1992,0.3


In [31]:
NDC.rename(columns={'string_field_0':'Pais'},inplace=True)
tabla = traductor(tabla)
tabla = filtro_latam(tabla)
tabla.head()

C:\Users\Francisco Cervantes\AppData\Local\Temp\ipykernel_5876\1093828181.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,Pais,Anio,Emisiones_de_CO2
0,Cuba,1988,34.538802
1,Cuba,1989,35.527976
2,Cuba,1990,35.625389
3,Cuba,1991,35.562074
4,Cuba,1992,32.389277


In [21]:
paises = tabla[tabla.Emisiones_de_CO2.isna()].Pais.value_counts().index
faltantes = tabla[tabla.Emisiones_de_CO2.isna()].Pais.value_counts().values
fig = px.bar(x = paises, y = faltantes, labels={'x':'', 'y':'Conteo'}, title='Cantidad de faltantes')
fig.show()

In [35]:
tabla2 = tabla[(tabla['Pais'] != 'Estados Unidos') & (tabla['Pais'] != 'Canadá') & (tabla['Anio'] == 2019)].copy()
fig = px.pie(tabla2, names='Pais', values='Emisiones_de_CO2', title = 'Emisiones de CO2 en 2019',  width=800, height=400)
fig.show()

In [11]:
NDC.rename(columns={'string_field_0':'Pais'},inplace=True)
tabla = traductor(tabla)
tabla = filtro_latam(tabla)
tabla.head()

C:\Users\Francisco Cervantes\AppData\Local\Temp\ipykernel_14004\1093828181.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.Pais[i]= NDC['Pais'][j]


,Pais,Anio,Emisiones_de_CO2
0,Cuba,1988,34.538802
1,Cuba,1989,35.527976
2,Cuba,1990,35.625389
3,Cuba,1991,35.562074
4,Cuba,1992,32.389277


In [12]:
tabla.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1920 entries, 0 to 1919
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Pais              1920 non-null   object 
 1   Anio              1920 non-null   Int64  
 2   Emisiones_de_CO2  1829 non-null   float64
dtypes: Int64(1), float64(1), object(1)
memory usage: 47.0+ KB


In [26]:
len(tabla[tabla.Pais == 'Surinam'])

160

In [34]:
len(tabla[tabla.Emisiones_de_CO2.isna()].Pais.value_counts())

29

In [37]:
tabla[tabla.Emisiones_de_CO2.isna()][tabla[tabla.Emisiones_de_CO2.isna()].Pais != 'Surinam'].Anio.max()

1992

## 0 - Columna 'Country'

In [5]:
# Cantidad de valores unicos
print(len(tabla['Country'].unique()))

231


In [6]:
# Cantidad de valores faltantes:
print(len(tabla['Country'][tabla['Country'].isna()]))

0


In [7]:
# Conteo de valores
tabla['Country'].value_counts()

World                   240
Netherlands Antilles    240
New Zealand             240
Nicaragua               240
Niger                   240
                       ... 
Germany, West           240
Ghana                   240
Gibraltar               240
Greece                  240
Zimbabwe                240
Name: Country, Length: 231, dtype: int64

In [8]:
# Verificamos si todos tienen 240
verification = True
for i in tabla['Country'].value_counts():
    if i != 240:
        verification = False
if verification: print("¡Todas las entidades se repiten 240 veces!")
else: print("¡Hay entidades con diferente cantidad de repeticiones a 240!")


¡Todas las entidades se repiten 240 veces!


## 1 - Columna 'Energy_type'

In [9]:
# Cantidad de valores únicos
print(len(tabla['Energy_type'].unique()))


6


In [10]:
# Cantidad de valores faltantes:
print(len(tabla['Energy_type'][tabla['Energy_type'].isna()]))

0


In [11]:
# Conteo de valores
tabla['Energy_type'].value_counts()

all_energy_types             9240
coal                         9240
natural_gas                  9240
petroleum_n_other_liquids    9240
nuclear                      9240
renewables_n_other           9240
Name: Energy_type, dtype: int64

## 2 - Columna 'Year'

In [12]:
# Cantidad de valores únicos
print(len(tabla['Year'].unique()))

40


In [13]:
# Cantidad de valores faltantes
print(len(tabla['Year'][tabla['Year'].isna()]))

0


In [14]:
# Conteo de valores
tabla['Year'].value_counts()

1980    1386
1981    1386
2002    1386
2003    1386
2004    1386
2005    1386
2006    1386
2007    1386
2008    1386
2009    1386
2010    1386
2011    1386
2012    1386
2013    1386
2014    1386
2015    1386
2016    1386
2017    1386
2018    1386
2001    1386
2000    1386
1999    1386
1989    1386
1982    1386
1983    1386
1984    1386
1985    1386
1986    1386
1987    1386
1988    1386
1990    1386
1998    1386
1991    1386
1992    1386
1993    1386
1994    1386
1995    1386
1996    1386
1997    1386
2019    1386
Name: Year, dtype: int64

In [15]:
# Verificamos si todos tienen 240
verification = True
for i in tabla['Year'].value_counts():
    if i != 1386:
        verification = False
if verification: print("¡Todas las entidades se repiten 1386 veces!")
else: print("¡Hay entidades con diferente cantidad de repeticiones a 1386!")

¡Todas las entidades se repiten 1386 veces!


## 3 - Columnas 'Energy_production' 'GDP' 'Population' 'Energy_intensity_per_capita' 'Energy_intensity_by_GDP' 'CO2_emission'

In [16]:
# Cantidad de valores nulos
total = len(tabla)
columnas = ['Energy_consumption','Energy_production','GDP','Population','Energy_intensity_per_capita','Energy_intensity_by_GDP','CO2_emission']
for columna in columnas:
    faltantes = len(tabla[columna][tabla[columna].isna()])
    print(columna, ':', faltantes, ':', faltantes / total, '%' )

Energy_consumption : 11153 : 0.20117243867243867 %
Energy_production : 11151 : 0.20113636363636364 %
GDP : 15414 : 0.278030303030303 %
Population : 9426 : 0.17002164502164502 %
Energy_intensity_per_capita : 5082 : 0.09166666666666666 %
Energy_intensity_by_GDP : 5082 : 0.09166666666666666 %
CO2_emission : 3826 : 0.06901154401154401 %


In [17]:
tabla_g = tabla[tabla['Country'] == 'World'][['Year', 'Energy_type', 'CO2_emission']].copy()
tabla_g = tabla_g[(tabla_g['Energy_type'] != 'all_energy_types')]
# tabla_g = tabla_g.groupby(['Year', 'Energy_type']).sum()
tabla_g.reset_index(inplace=True)
fig = px.line(tabla_g, 'Year', 'CO2_emission', color='Energy_type', title='Emisiones CO2 por tipo de energía en el mundo')
fig.show()


In [18]:
tabla_g = tabla[tabla['Country'] == 'World'][['Year', 'Energy_type', 'Energy_production']].copy()
tabla_g = tabla_g[(tabla_g['Energy_type'] != 'all_energy_types')]
# tabla_g = tabla_g.groupby(['Year', 'Energy_type']).sum()
tabla_g.reset_index(inplace=True)
fig = px.line(tabla_g, 'Year', 'Energy_production', color='Energy_type', title='Producción de energía por tipo de energía en el mundo')
fig.show()

In [19]:
tabla_g = tabla[tabla['Country'] == 'World'][['Year', 'Energy_type', 'Energy_consumption']].copy()
tabla_g = tabla_g[(tabla_g['Energy_type'] != 'all_energy_types')]
# tabla_g = tabla_g.groupby(['Year', 'Energy_type']).sum()
tabla_g.reset_index(inplace=True)
fig = px.line(tabla_g, 'Year', 'Energy_consumption', color='Energy_type', title='Producción de energía por tipo de energía en el mundo')
fig.show()

In [20]:
tabla_g = tabla[tabla['Country'] == 'World'][['Year', 'Energy_intensity_per_capita','Energy_intensity_by_GDP']].copy()
tabla_g.reset_index(inplace=True)
fig = px.line(tabla_g, 'Year', 'Energy_intensity_per_capita', title='Intensidad de energía per cápita')
fig.show()

In [21]:
tabla_g = tabla[tabla['Country'] == 'World'][['Year', 'Energy_intensity_by_GDP']].copy()
tabla_g.reset_index(inplace=True)
fig = px.line(tabla_g, 'Year', 'Energy_intensity_by_GDP', title='Intensidad de energía por PIB')
fig.show()

----
## Observaciones y Recomendaciones:

1) Los registros se repiten 5 veces (Habiendo 6 registros con los mismos valores) debido a las columnas 'Energy_type', 'CO2_emission', 'Energy_consumption' y 'Energy_production' por lo que para evitar esto, se podría usar otra tabla con los valores repetidos para cada año

In [22]:
tabla.head(12)

,Country,Energy_type,Year,Energy_consumption,Energy_production,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP,CO2_emission
0,World,all_energy_types,1980,292.899790,296.337228,27770.910281,4.298127e+06,68.145921,10.547,4946.627130
1,World,coal,1980,78.656134,80.114194,27770.910281,4.298127e+06,68.145921,10.547,1409.790188
2,World,natural_gas,1980,53.865223,54.761046,27770.910281,4.298127e+06,68.145921,10.547,1081.593377
3,World,petroleum_n_other_liquids,1980,132.064019,133.111109,27770.910281,4.298127e+06,68.145921,10.547,2455.243565
4,World,nuclear,1980,7.575700,7.575700,27770.910281,4.298127e+06,68.145921,10.547,0.000000
5,World,renewables_n_other,1980,20.702344,20.775178,27770.910281,4.298127e+06,68.145921,10.547,0.000000
6,Afghanistan,all_energy_types,1980,0.026583,0.072561,NaN,1.335650e+04,1.990283,0.000,NaN
7,Afghanistan,coal,1980,0.002479,0.002355,NaN,1.335650e+04,1.990283,0.000,NaN
8,Afghanistan,natural_gas,1980,0.002094,0.062820,NaN,1.335650e+04,1.990283,0.000,NaN
9,Afghanistan,petroleum_n_other_liquids,1980,0.014624,0.000000,NaN,1.335650e+04,1.990283,0.000,NaN


In [23]:
# Tabla de una dimensión
tabla_1 = tabla[['Country', 'Year', 'GDP', 'Population', 'Energy_intensity_per_capita', 'Energy_intensity_by_GDP']].copy()
tabla_1.drop_duplicates(inplace = True)
tabla_1.reset_index(drop= True, inplace = True)
tabla_1.head()


,Country,Year,GDP,Population,Energy_intensity_per_capita,Energy_intensity_by_GDP
0,World,1980,27770.910281,4.298127e+06,68.145921,10.547
1,Afghanistan,1980,NaN,1.335650e+04,1.990283,0.000
2,Albania,1980,NaN,2.682700e+03,60.752906,0.000
3,Algeria,1980,NaN,1.922170e+04,40.615303,0.000
4,American Samoa,1980,NaN,3.264600e+01,180.515604,0.000


In [24]:
# Tabla de hechos
tabla_2 = tabla[['Country', 'Year', 'Energy_type', 'Energy_consumption', 'Energy_production', 'CO2_emission']].copy()
tabla_2.head()

,Country,Year,Energy_type,Energy_consumption,Energy_production,CO2_emission
0,World,1980,all_energy_types,292.899790,296.337228,4946.627130
1,World,1980,coal,78.656134,80.114194,1409.790188
2,World,1980,natural_gas,53.865223,54.761046,1081.593377
3,World,1980,petroleum_n_other_liquids,132.064019,133.111109,2455.243565
4,World,1980,nuclear,7.575700,7.575700,0.000000


Comparamos los espacios de las tablas con la original

In [25]:
tabla.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55440 entries, 0 to 55439
Data columns (total 10 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Country                      55440 non-null  object 
 1   Energy_type                  55440 non-null  object 
 2   Year                         55440 non-null  int64  
 3   Energy_consumption           44287 non-null  float64
 4   Energy_production            44289 non-null  float64
 5   GDP                          40026 non-null  float64
 6   Population                   46014 non-null  float64
 7   Energy_intensity_per_capita  50358 non-null  float64
 8   Energy_intensity_by_GDP      50358 non-null  float64
 9   CO2_emission                 51614 non-null  float64
dtypes: float64(7), int64(1), object(2)
memory usage: 4.7+ MB


In [26]:
tabla_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9240 entries, 0 to 9239
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Country                      9240 non-null   object 
 1   Year                         9240 non-null   int64  
 2   GDP                          6671 non-null   float64
 3   Population                   7669 non-null   float64
 4   Energy_intensity_per_capita  8393 non-null   float64
 5   Energy_intensity_by_GDP      8393 non-null   float64
dtypes: float64(4), int64(1), object(1)
memory usage: 433.2+ KB


In [27]:
tabla_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55440 entries, 0 to 55439
Data columns (total 6 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Country             55440 non-null  object 
 1   Year                55440 non-null  int64  
 2   Energy_type         55440 non-null  object 
 3   Energy_consumption  44287 non-null  float64
 4   Energy_production   44289 non-null  float64
 5   CO2_emission        51614 non-null  float64
dtypes: float64(3), int64(1), object(2)
memory usage: 3.0+ MB


De esta manera se ahorran más de 1 MB de espacio